In [1]:
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa
from tensorflow.keras.layers import Layer

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Pre-Processing

In [2]:

MONET_FILENAMES = tf.io.gfile.glob("../images/artists/monet/monet_tfrec/*.tfrec")
print('Monet TFRecord Files:', len(MONET_FILENAMES))

PHOTO_FILENAMES = tf.io.gfile.glob("../images/random_images/photo_tfrec/*.tfrec")
print('Photo TFRecord Files:', len(PHOTO_FILENAMES))

Monet TFRecord Files: 5
Photo TFRecord Files: 20


In [ ]:
def random_crop(image):
    cropped_image = tf.image.random_crop(image, size=[SIZE_RESIZE, SIZE_RESIZE, 3])
    return cropped_image